# Buenos Aires , where to open more healthy lifestyle choices

## Author: Alejandro Vidaurri

### Date: Jun 1st 2021


The problem we'll try to solve is identify the neighborhoods that would be good options to open vegan restaurants in Buenos Aires Capital Federal (BSAS). 

Buenos aires has been tough on fast food restaurants, check the names of the pizza restaurants and you would not find the franchise names mentioned. On the other side other fast food restaurants such as McDonalds and Strarbucks have positioned and have some market share. 



Using geographical data from Google maps, we'll find the number of traditional restaurants (asado and pizzerias) and also non traditional ones (mcdonalds and starbucks) and use them to find clusters . Once we can cluster similar neighborhoods we'll check which neighborhoods have already accepted vegan restaurants and extrapolate to the neighbohoods in such cluster as good options for more vegan restaurants

eg: Chacarita and Puerto Madero BSAS neihborhoods end in the same cluster and Puerto Madero already has accepted non Vegan restaurants, then Chacarita is a good option to open a Vegan restaurant

The data will be provided by Wikipedia to find all the BSAS neighborhoods, then using google maps we'll find the Latitude and Longitude of each of them.  From there a 500m radius will be used to gather the restaurants per name, such as: pizzeria, asado, mcdonalds, starbucks. Each of these findings will be turned into a dimension using one hot encoding, then a normalized frequency and finaly a dimension to run KMeans algorithm

In [ ]:
API_KEY = 'REMOVED'
fast_food = ['mcdonalds','starbucks']
traditional = ['asado','pizzeria']
for type_of_food in fast_food:
    url='https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=-34.60598089051447,-58.45093997972278&radius=30000&key={}&keyword={}'.format(API_KEY, type_of_food)
    print(url)
for type_of_food in traditional:
    url='https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=-34.60598089051447,-58.45093997972278&radius=30000&key={}&keyword={}'.format(API_KEY, type_of_food)
    print(url)

### Find the neighborhoods in buenos aires from Wikipedia

In [3]:
wiku_url  = 'https://en.wikipedia.org/wiki/Neighbourhoods_of_Buenos_Aires'
import requests
from bs4 import BeautifulSoup
import pandas as pd
response = requests.get(wiku_url)

soup = BeautifulSoup(response.text)

In [4]:
table = soup.find(name='table')

table_contents = []
for row in table.findAll('tr'):
    cell = {}
    tds = row.findAll('td')
    if len(tds) > 0:
        cell['Name'] = tds[0].text.replace('\n','')
        cell['Area in km2'] = tds[1].text.replace('\n','')
        cell['Population'] = tds[2].text.replace('\n','')
        cell['Communa'] = tds[3].text.replace('\n','')
        table_contents.append(cell)

df=pd.DataFrame(table_contents)

In [76]:
df.head()

,Name,Area in km2,Population,Communa
0,Agronomía,2.1,"13,963",15
1,Almagro,4.1,"128,206",5
2,Balvanera,4.4,"137,521",3
3,Barracas,7.6,"73,377",4
4,Belgrano,6.8,"126,816",13


### Build location dataframe with Geo Coordinates

In [6]:
location_list = []
site = 'Flores'
sample_loc_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=-34.60598089051447,-58.45093997972278&radius=30000&key={}&name={}%20Buenos%20Aires,%20Argentina'.format(API_KEY, site)
results = requests.get(sample_loc_url).json()['results']
location_list.append([(
    v['name'], 
    v['geometry']['location']['lat'], 
    v['geometry']['location']['lng'],  
    v['types'][0]) for v in results])

In [7]:
location_list

[[('Flores, Buenos Aires', -34.6374837, -58.4601452, 'sublocality_level_1')]]

In [8]:
location_list = []
for site in df['Name']:
    sample_loc_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=-34.60598089051447,-58.45093997972278&radius=30000&key={}&name={}%20Buenos%20Aires,%20Argentina'.format(API_KEY, site)
    results = requests.get(sample_loc_url).json()['results']
    location_list.append([(
    v['name'], 
    v['geometry']['location']['lat'], 
    v['geometry']['location']['lng'],
        
    v['geometry']['viewport']['northeast']['lat'], 
    v['geometry']['viewport']['northeast']['lng'], 
        
    v['geometry']['viewport']['southwest']['lat'], 
    v['geometry']['viewport']['southwest']['lng'],
    v['types'][0]) for v in results])
    
    


In [78]:
location_df = pd.DataFrame([item for locations_list in location_list for item in locations_list])
location_df.columns = ['Neighborhood', 
          'Latitude', 
          'Longitude',
           'Northeast Latitude', 
          'Northeast Longitude',
           'Southwest Latitude', 
          'Southwest Longitude', 
            'Location type']
location_df.head()

,Neighborhood,Latitude,Longitude,Northeast Latitude,Northeast Longitude,Southwest Latitude,Southwest Longitude,Location type
0,Agronomía,-34.595040,-58.494292,-34.584818,-58.475882,-34.599671,-58.503472,sublocality_level_1
1,Almagro,-34.611445,-58.420994,-34.597703,-58.411919,-34.622074,-58.433334,sublocality_level_1
2,Balvanera,-34.610122,-58.405941,-34.598003,-58.391659,-34.620642,-58.414660,sublocality_level_1
3,Barracas,-34.645404,-58.381284,-34.626647,-58.367636,-34.662402,-58.401502,sublocality_level_1
4,Belgrano,-34.562084,-58.456686,-34.535931,-58.428204,-34.575008,-58.473368,sublocality_level_1


### Map of Neighborhoods in Buenos Aires

I would need to find a way to place each business in a neighborhood that is more accurate.
In this map the actual radius and a 500 meter radius is placed on top of each neighborhood
For now I'll use the 500 meter radius

In [25]:
from haversine import haversine, Unit # to find distances from lat lon

latitude=location_df['Latitude'].mean()
longitude = location_df['Longitude'].mean()
import folium # map rendering library

# create map of New York using latitude and longitude values
BSAS = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, neighborhood, nelat, nelng, swlat, swlng in zip(location_df['Latitude'], location_df['Longitude'], 
                                  location_df['Neighborhood'], 
                                  location_df['Northeast Latitude'], location_df['Northeast Longitude'],
                                  location_df['Southwest Latitude'], location_df['Southwest Longitude']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    #folium.CircleMarker(
    #    [lat, lng],
    #    radius=1,
    #    color='blue',
    #    fill=True,
    #    fill_color='#3186cc',
    #    opacity=0.7,
    #    parse_html=False).add_to(BSAS)  

    #folium.Polygon(
    #    [[swlat, nelng], [nelat, nelng], [nelat, swlng], [swlat, swlng]],
    #    color='#FF0000F0',
    #    fill=True,
    #    fill_color='#FF0000F0',
    #    fill_opacity=0.1,
    #    color_opacity=0.1,
    #    parse_html=False).add_to(BSAS) 

    radius = min(haversine( (nelat, (swlng + nelng)/2), (lat, lng) , unit=Unit.METERS), haversine( ((nelat+swlat)/2, nelng), (lat, lng) , unit=Unit.METERS))
    folium.Circle(
        [lat, lng],
        radius=radius, 
        color='#00AF00A0',
        tooltip = 'Real radius {:.2f}m<br/>Neighborhood: {}'.format(radius, neighborhood),
        fill=True,
        fill_color='#00FF00A0',
         
        parse_html=False).add_to(BSAS)  
    
    folium.Circle(
        [lat, lng],
        radius=500, 
        color='#0000AFA0',
        tooltip = '500m radius</br>Neighborhood: {}'.format( neighborhood),
        fill=True,
        fill_color='#0000AFA0',
        fill_opacity=0.7,
        parse_html=False).add_to(BSAS)  

    
BSAS

## Get Venues

In [26]:
import pandas as pd
def getNearbyVenues(names, latitudes, longitudes, keywords, radius=500):
    
    venues_list=[]
    for name, lat, lon in zip(names, latitudes, longitudes):
            
        # create the API request URL
        #url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        #    CLIENT_ID, 
        #    CLIENT_SECRET, 
        #    VERSION, 
        #    lat, 
        #    lng, 
        #    radius, 
        #    LIMIT)
        for k in keywords :
                url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/{}?location={},{}&radius={}&key={}&keyword={}'.format(
                    RETURN_TYPE,  lat, lon, radius, API_KEY, k, 
                    )
                print(url)
                # make the GET request
                results = requests.get(url).json()['results']
                #results = pd.json_normalize(results) # flatten JSON
                #print( v['geometry.location.lat'], v['geometry.location.lat'])
                # return only relevant information for each nearby venue
                venues_list.append([(
                    name, 
                    lat, 
                    lng, 
                    v['name'], 
                    v['geometry']['location']['lat'], 
                    v['geometry']['location']['lng'],  
                    v['types'][0], k) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category',
                  'venue type']
    
    return(nearby_venues)

In [ ]:
Vegan_Venues  = getNearbyVenues(location_df.loc[:, 'Neighborhood'],
               location_df.loc[:, 'Latitude'],
                location_df.loc[:, 'Longitude'],['vegan']
               )

In [82]:
Buenos_Aires_Venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,venue type
0,Agronomía,-34.595040,-58.491458,EL JAGÜEL,-34.597363,-58.490994,restaurant,pizzeria
1,Agronomía,-34.595040,-58.491458,Pizzería Nay,-34.597546,-58.492421,restaurant,pizzeria
2,Agronomía,-34.595040,-58.491458,Pizzería La Tía,-34.601455,-58.489739,restaurant,pizzeria
3,Agronomía,-34.595040,-58.491458,Heroes Pizza On Fire,-34.591642,-58.501313,restaurant,pizzeria
4,Agronomía,-34.595040,-58.491458,Kentucky,-34.596429,-58.496925,restaurant,pizzeria
...,...,...,...,...,...,...,...,...
548,Villa Urquiza,-34.570518,-58.491458,La Farola de Urquiza,-34.574015,-58.485534,restaurant,pizzeria
549,Villa Urquiza,-34.570518,-58.491458,La Ruedita Pizzeria,-34.576408,-58.492928,restaurant,pizzeria
550,Villa Urquiza,-34.570518,-58.491458,Pizza Vegana Villa Urquiza,-34.568298,-58.486046,restaurant,pizzeria
551,Villa Urquiza,-34.570518,-58.491458,Kentucky,-34.575723,-58.484320,bar,pizzeria


In [49]:
print(Buenos_Aires_Venues.shape)
Buenos_Aires_Venues.head()



(553, 8)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,venue type
0,Agronomía,-34.59504,-58.491458,EL JAGÜEL,-34.597363,-58.490994,restaurant,pizzeria
1,Agronomía,-34.59504,-58.491458,Pizzería Nay,-34.597546,-58.492421,restaurant,pizzeria
2,Agronomía,-34.59504,-58.491458,Pizzería La Tía,-34.601455,-58.489739,restaurant,pizzeria
3,Agronomía,-34.59504,-58.491458,Heroes Pizza On Fire,-34.591642,-58.501313,restaurant,pizzeria
4,Agronomía,-34.59504,-58.491458,Kentucky,-34.596429,-58.496925,restaurant,pizzeria


In [50]:
print(Vegan_Venues.shape)
Vegan_Venues.head()

(50, 8)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,venue type
0,Agronomía,-34.595040,-58.491458,vida sana,-34.598114,-58.498929,restaurant,vegan
1,Belgrano,-34.562084,-58.491458,Vica,-34.563372,-58.459650,restaurant,vegan
2,Belgrano,-34.562084,-58.491458,JU. Vegetariano y vegano,-34.557728,-58.449664,grocery_or_supermarket,vegan
3,Belgrano,-34.562084,-58.491458,Heladería Bali Kosher & Vegan Belgrano,-34.559794,-58.462135,restaurant,vegan
4,Belgrano,-34.562084,-58.491458,Ju Detox,-34.557724,-58.449619,restaurant,vegan


In [51]:
Buenos_Aires_Venues.groupby('Neighborhood')[['Venue']].count()

,Venue
Neighborhood,
Agronomía,8
Almagro,16
Balvanera,14
Barracas,5
Belgrano,21
Boedo,13
Caballito,16
Chacarita,11
Coghlan,7


In [52]:
Vegan_Venues.groupby('Neighborhood')[['Venue']].count()

,Venue
Neighborhood,
Agronomía,1
Belgrano,4
Caballito,2
Chacarita,1
Coghlan,2
Colegiales,2
"Flores, Buenos Aires",1
Monserrat,4
Monte Castro,1


In [53]:
# one hot encoding
buenos_aires_onehot = pd.get_dummies(Buenos_Aires_Venues[['venue type']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
buenos_aires_onehot['Neighborhood'] = Buenos_Aires_Venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [buenos_aires_onehot.columns[-1]] + list(buenos_aires_onehot.columns[:-1])
buenos_aires_onehot = buenos_aires_onehot[fixed_columns]

buenos_aires_onehot.head()

,Neighborhood,asado,mcdonalds,pizzeria,starbucks
0,Agronomía,0,0,1,0
1,Agronomía,0,0,1,0
2,Agronomía,0,0,1,0
3,Agronomía,0,0,1,0
4,Agronomía,0,0,1,0


In [133]:
bsas_grouped = buenos_aires_onehot.groupby('Neighborhood').mean().reset_index()
bsas_grouped

,Neighborhood,asado,mcdonalds,pizzeria,starbucks
0,Agronomía,0.000000,0.125000,0.875000,0.000000
1,Almagro,0.187500,0.000000,0.750000,0.062500
2,Balvanera,0.214286,0.214286,0.500000,0.071429
3,Barracas,0.400000,0.000000,0.600000,0.000000
4,Belgrano,0.190476,0.095238,0.619048,0.095238
5,Boedo,0.076923,0.000000,0.923077,0.000000
6,Caballito,0.125000,0.062500,0.687500,0.125000
7,Chacarita,0.272727,0.090909,0.636364,0.000000
8,Coghlan,0.285714,0.000000,0.714286,0.000000
9,Colegiales,0.153846,0.153846,0.615385,0.076923


In [107]:
#bsas_grouped= bsas_grouped.append(pd.DataFrame([['La Paternal', 0,0,0,0]], columns=('Neighborhood','asado','mcdonalds','pizzeria','starbucks')), ignore_index=True  )

In [114]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [117]:
import numpy as np
num_top_venues = 4

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bsas_grouped['Neighborhood']

for ind in np.arange(bsas_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bsas_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
0,Agronomía,pizzeria,mcdonalds,starbucks,asado
1,Almagro,pizzeria,asado,starbucks,mcdonalds
2,Balvanera,pizzeria,mcdonalds,asado,starbucks
3,Barracas,pizzeria,asado,starbucks,mcdonalds
4,Belgrano,pizzeria,asado,starbucks,mcdonalds
5,Boedo,pizzeria,asado,starbucks,mcdonalds
6,Caballito,pizzeria,starbucks,asado,mcdonalds
7,Chacarita,pizzeria,asado,mcdonalds,starbucks
8,Coghlan,pizzeria,asado,starbucks,mcdonalds
9,Colegiales,pizzeria,mcdonalds,asado,starbucks


In [164]:
from sklearn.cluster import KMeans

kclusters = 12

bsas_grouped_clustering = bsas_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bsas_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 



array([11,  5, 10,  6,  0,  1,  0,  4,  4,  0], dtype=int32)

In [165]:
neighborhoods_venues_sorted.head()
neighborhoods_venues_sorted.drop(axis=1, columns=['Cluster Labels'], inplace=True)

In [166]:

# add clustering label
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

BSAS_merged = location_df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
BSAS_merged = BSAS_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

BSAS_merged # check the last columns!

,Neighborhood,Latitude,Longitude,Northeast Latitude,Northeast Longitude,Southwest Latitude,Southwest Longitude,Location type,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
0,Agronomía,-34.595040,-58.494292,-34.584818,-58.475882,-34.599671,-58.503472,sublocality_level_1,11.0,pizzeria,mcdonalds,starbucks,asado
1,Almagro,-34.611445,-58.420994,-34.597703,-58.411919,-34.622074,-58.433334,sublocality_level_1,5.0,pizzeria,asado,starbucks,mcdonalds
2,Balvanera,-34.610122,-58.405941,-34.598003,-58.391659,-34.620642,-58.414660,sublocality_level_1,10.0,pizzeria,mcdonalds,asado,starbucks
3,Barracas,-34.645404,-58.381284,-34.626647,-58.367636,-34.662402,-58.401502,sublocality_level_1,6.0,pizzeria,asado,starbucks,mcdonalds
4,Belgrano,-34.562084,-58.456686,-34.535931,-58.428204,-34.575008,-58.473368,sublocality_level_1,0.0,pizzeria,asado,starbucks,mcdonalds
5,Boedo,-34.630538,-58.419221,-34.620642,-58.410489,-34.640200,-58.427795,sublocality_level_1,1.0,pizzeria,asado,starbucks,mcdonalds
6,Caballito,-34.615924,-58.440603,-34.602673,-58.426756,-34.630639,-58.462705,sublocality_level_1,0.0,pizzeria,starbucks,asado,mcdonalds
7,Chacarita,-34.585999,-58.454436,-34.578294,-58.438699,-34.597835,-58.466958,sublocality_level_1,4.0,pizzeria,asado,mcdonalds,starbucks
8,Coghlan,-34.560159,-58.471566,-34.553534,-58.466576,-34.566849,-58.483560,sublocality_level_1,4.0,pizzeria,asado,starbucks,mcdonalds
9,Colegiales,-34.576023,-58.448355,-34.566108,-58.439939,-34.583192,-58.463355,sublocality_level_1,0.0,pizzeria,mcdonalds,asado,starbucks


In [167]:
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[BSAS_merged['Latitude'].mean(), BSAS_merged['Longitude'].mean()], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(BSAS_merged['Latitude'], BSAS_merged['Longitude'], 
                                  BSAS_merged['Neighborhood'], BSAS_merged['Cluster Labels']):
    if not np.isnan( cluster):
        #label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            tooltip=str(poi) + ' Cluster ' + str(cluster),
            color=rainbow[int(cluster-1)],
            fill=True,
            fill_color=rainbow[int(cluster-1)],
            fill_opacity=0.7).add_to(map_clusters)
        
map_clusters

In [168]:
BSAS_merged.loc[BSAS_merged['Cluster Labels'] == 9, BSAS_merged.columns[[0] + list(range(5, BSAS_merged.shape[1]))]]

,Neighborhood,Southwest Latitude,Southwest Longitude,Location type,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
28,Retiro,-34.599293,-58.387910,sublocality_level_1,9.0,starbucks,pizzeria,asado,mcdonalds
31,San Nicolás,-34.609533,-58.392931,sublocality_level_1,9.0,pizzeria,starbucks,asado,mcdonalds


In [169]:
BSAS_merged.loc[BSAS_merged['Cluster Labels'] == 3, BSAS_merged.columns[[0] + list(range(5, BSAS_merged.shape[1]))]]

,Neighborhood,Southwest Latitude,Southwest Longitude,Location type,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
17,Monserrat,-34.618159,-58.391951,sublocality_level_1,3.0,pizzeria,asado,starbucks,mcdonalds
21,Palermo,-34.597855,-58.449044,sublocality_level_1,3.0,pizzeria,asado,mcdonalds,starbucks
27,Recoleta,-34.599760,-58.416000,sublocality_level_1,3.0,pizzeria,asado,starbucks,mcdonalds


In [182]:
BSAS_merged.loc[BSAS_merged['Cluster Labels'] == 7, BSAS_merged.columns[[0] + list(range(5, BSAS_merged.shape[1]))]]

,Neighborhood,Southwest Latitude,Southwest Longitude,Location type,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
20,"Núñez, Buenos Aires",-34.558928,-58.475756,sublocality_level_1,7.0,pizzeria,asado,mcdonalds,starbucks
30,San Cristóbal,-34.630356,-58.412544,sublocality_level_1,7.0,pizzeria,asado,mcdonalds,starbucks
32,San Telmo,-34.629494,-58.377425,sublocality_level_1,7.0,asado,pizzeria,starbucks,mcdonalds


In [181]:
Vegan_Venues.groupby('Neighborhood')[['Venue']].count().sort_values(by='Venue',ascending=False)

,Venue
Neighborhood,
San Nicolás,7
San Telmo,6
Monserrat,4
Palermo,4
Belgrano,4
Villa del Parque,2
Caballito,2
Coghlan,2
Colegiales,2


With the data gathered Clusters 9,3 and 7 are the most likely to accept new restaurants like Vegan

Cluster 3 has the most already (montserrat=4, Palermo=4) and likely neighborhood Retiro in cluster 9 could accept more as San Nicolas has already the most by individual Neighborhood (7)

San Telmo in cluster 7 has the second place in number ov vegan restaurants, therefore San Cristobal and Nuñez could also be good options for more vegan restaurants